In [160]:
%display latex

In [161]:
p = 3
N = 2

F = GF(p^N, 'x')
x = F.gen()

In [185]:
basis = [F(1), x + 1] # almost self-dual

def toInt(k):
    return list(F).index(k)

def components(k):
    return vector([(k * b).trace() for b in basis])

omega = exp(2 * pi * I / p)
chi = lambda k: omega^int(k.trace())

In [186]:
for bj in basis:
    for bk in basis:
        print(f'{bj}\t {bk} \t-> {(bj * bk).trace()}')

1	 1 	-> 2
1	 x + 1 	-> 0
x + 1	 1 	-> 0
x + 1	 x + 1 	-> 1


In [187]:
def qj(j):
    return (F(2) - F(1)) * int(j == 0) + F(1)

In [188]:
# verify that it is almost self-dual
for i, bi in enumerate(basis):
    for j, bj in enumerate(basis):
        lhs = (bi * bj).trace()
        rhs = qj(j) * F(int(i == j))
        if lhs != rhs:
            raise Exception(bi, bj, lhs, rhs)

In [189]:
# is the char additive?
for j in F:
    for k in F:
        if chi(j) * chi(k) != chi(j + k):
            raise Exception

In [190]:
# factorization of the character 
for a in F:
    for b in F:
        ais = components(a)
        bis = components(b)
        lhs = chi(a * b)
        rhs = 1
        for i in range(N):
            rhs *= omega^int(ais[i] * bis[i] * qj(i))
        if lhs != rhs:
            print(f'{lhs}\t {rhs}')

In [191]:
def Phi(a, b):
    return chi(-F(2)^-1 * a * b)

In [192]:
def DPS(curve):
    ps = zero_matrix(SR, p^N, p^N)
    ps[0,:] = 1
    for mu in F:
        for j, a in enumerate(F):
            b = curve(mu)(a)
            ps[j, toInt(b)] = Phi(a, b)
    return ps

In [193]:
def ray(mu):
    return lambda k: mu * k

In [194]:
ps = DPS(ray)
ps

[                      1                       1                       1                       1                       1                       1                       1                       1                       1]
[                      1                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2]
[                      1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1]
[                      1     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2]
[                      1 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2]
[                      1                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2]
[                      1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1]
[                      1 (1/2*I*sqrt(3) - 1/2)^2     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2]
[                      1     1/2*I*sqrt(3) - 1/2                       1     1/2*I*sqrt(3) - 1/2     1/2*I*sqrt(3) - 1/2 (1/2*I*sqrt(3) - 1/2)^2                       1 (1/2*I*sqrt(3) - 1/2)^2 (1/2*I*sqrt(3) - 1/2)^2]

In [176]:
w = var('omega')

ps = DPS(ray)
ps

[      1       1       1       1       1       1       1       1       1]
[      1       1   omega   omega omega^2       1 omega^2 omega^2   omega]
[      1   omega   omega omega^2       1 omega^2 omega^2   omega       1]
[      1   omega omega^2       1 omega^2 omega^2   omega       1   omega]
[      1 omega^2       1 omega^2 omega^2   omega       1   omega   omega]
[      1       1 omega^2 omega^2   omega       1   omega   omega omega^2]
[      1 omega^2 omega^2   omega       1   omega   omega omega^2       1]
[      1 omega^2   omega       1   omega   omega omega^2       1 omega^2]
[      1   omega       1   omega   omega omega^2       1 omega^2 omega^2]

In [206]:
# verify that the phase satisfies the recurrence relation
def VerifyRecRel(curve=ray):
    for mu in F:
        for t1 in F:
            for t2 in F:
                lhs = Phi(t1, curve(mu)(t1)) * Phi(t2, curve(mu)(t2))
                rhs = Phi(t1 + t2, curve(mu)(t1 + t2)) * chi(t2 * curve(mu)(t1))
                if lhs != rhs:
                    raise Exception
    return True

In [207]:
VerifyRecRel(ray)

True